## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2023-07-25 12:18:25.975580: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop('EIN', axis = 1, inplace = True)
application_df.drop('NAME', axis = 1, inplace = True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [9]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [10]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = [index for index,row in application_counts.items() if row < 500]


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [14]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
counts_greater_than_1 = classification_counts[classification_counts > 1]
counts_greater_than_1

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [15]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = [index for index,row in classification_counts.items() if row < classification_counts.mean()]


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [16]:
# Convert categorical data to numeric with `pd.get_dummies`
columns_to_convert = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
                      'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']
processed_df = pd.get_dummies(application_df, columns=columns_to_convert, drop_first=True)


In [19]:
# Split our preprocessed data into our features and target arrays
X = processed_df
y = processed_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [24]:
print(X_train.shape)
print(X_test.shape)

(25724, 38)
(8575, 38)


## Compile, Train and Evaluate the Model

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=10, activation="tanh", input_dim=38))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="tanh"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                390       
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 487 (1.90 KB)
Trainable params: 487 (1.90 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.1566 - accuracy: 0.9584
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.0077 - accuracy: 0.9992
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.0034 - accuracy: 0.9993
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.0020 - accuracy: 0.9996
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.0013 - accuracy: 0.9997
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.0010 - accuracy: 0.9997
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 7.3288e-04 - accuracy: 0.9997
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 8.4672e-04 - accuracy: 0.9997
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 3.3645e-04 - accuracy: 0.9998
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 2.6

804/804 [==============================] - 1s 1ms/step - loss: 7.9414e-10 - accuracy: 1.0000
Epoch 79/100
804/804 [==============================] - 1s 1ms/step - loss: 7.7593e-10 - accuracy: 1.0000
Epoch 80/100
804/804 [==============================] - 1s 1ms/step - loss: 7.5542e-10 - accuracy: 1.0000
Epoch 81/100
804/804 [==============================] - 1s 1ms/step - loss: 7.3756e-10 - accuracy: 1.0000
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 7.2237e-10 - accuracy: 1.0000
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 7.0846e-10 - accuracy: 1.0000
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 6.9259e-10 - accuracy: 1.0000
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 6.7987e-10 - accuracy: 1.0000
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 6.6604e-10 - accuracy: 1.0000
Epoch 87/100
804/804 [==============================] - 1s 

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 3.8704e-07 - accuracy: 1.0000 - 347ms/epoch - 1ms/step
Loss: 3.8704064309058595e-07, Accuracy: 1.0


In [32]:
# Export our model to HDF5 file
nn_model.save("nn_model.h5")